# Modulo 9 – NLP Avanzato, Deep Learning per Sequenze e Progetti Finali
Questo notebook riassume i concetti avanzati di NLP con Transformers, modelli RNN/GRU/seq2seq e descrive i tre progetti finali end‑to‑end del corso.

## 1. NLP Avanzato con Transformers
- Tokenizzazione subword (BPE, WordPiece)
- Embedding contestuali (DistilBERT/BERT)
- Fine-tuning per classificazione testo

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

MODEL = './hf/distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModel.from_pretrained(MODEL)

txt = 'Machine learning con Python è fantastico!'
enc = tokenizer(txt, return_tensors='pt')
with torch.no_grad():
    out = model(**enc)
cls_embed = out.last_hidden_state[:,0,:]
cls_embed.shape

## 2. Deep Learning per Sequenze (RNN, GRU)
Esempio minimale di classificazione positiva/negativa con GRU.

In [ ]:
import torch
from torch import nn, optim

sent = ['oggi è bello', 'odio la pioggia']
lab = [1,0]
w2i={'<pad>':0,'<unk>':1,'oggi':2,'è':3,'bello':4,'odio':5,'la':6,'pioggia':7}
def enc(s):
    ids=[w2i.get(w,1) for w in s.split()];
    ids += [0]*(4-len(ids)); return ids
X=torch.tensor([enc(s) for s in sent]); y=torch.tensor(lab)

class GRU(nn.Module):
    def __init__(self):
        super().__init__();
        self.emb=nn.Embedding(len(w2i),16,padding_idx=0);
        self.gru=nn.GRU(16,32,batch_first=True);
        self.fc=nn.Linear(32,2)
    def forward(self,x):
        e=self.emb(x); _,h=self.gru(e); return self.fc(h[-1])

m=GRU(); opt=optim.Adam(m.parameters(),lr=1e-3); lossF=nn.CrossEntropyLoss()
for _ in range(30):
    opt.zero_grad(); out=m(X); loss=lossF(out,y); loss.backward(); opt.step()
print('Pred:', m(X).argmax(1))

## 3. Seq2Seq – Mini esempio "string reverse"
Dimostrazione concettuale di encoder–decoder.

In [ ]:
# toy seq2seq: inverti stringhe
pairs=[('ciao','oaic'),('mare','eram')]
chars=sorted({c for a,b in pairs for c in a+b})
stoi={'<pad>':0,'<sos>':1,'<eos>':2}
for c in chars: stoi[c]=len(stoi)
itos={i:s for s,i in stoi.items()}

def enc(s): return torch.tensor([stoi[c] for c in s],dtype=torch.long)

import torch
from torch import nn,optim
# Nota: implementazione completa seq2seq nel file separato del modulo

## 4. Progetti Finali End‑to‑End
### A) Intent + FAQ Bot (Transformers + RAG)
- Fine-tuning per intent
- RAG su FAQ/manuale
- Chat web consolidata (Flask)

### B) Previsione Sequenze con GRU
- Serie temporali o testo
- GRU/LSTM
- API + dashboard

### C) Assistente AI Multimodale (Testo + Voce + RAG)
- MediaRecorder
- STT + RAG + TTS
- Chat UI avanzata